# Importing the Libraries

In [3]:
#Importing the libraries to watch the 'fits' image and get the data array
import astropy
#import plotly.graph_objects as go
from astropy.io import fits
from sklearn.metrics import mean_squared_error
#Importing a library that is useful to read the original file
import pandas as pd
import pylab as plb
from datetime import datetime

import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp
#Importing a visual library with some illustrative set up
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors 
from matplotlib import cm
import numpy as np
import math 
import pywt
import seaborn as sns
plt.style.use('fivethirtyeight')
plt.rcParams['font.family'] = 'sans-serif' 
plt.rcParams['font.serif'] = 'Ubuntu' 
plt.rcParams['font.monospace'] = 'Ubuntu Mono' 
plt.rcParams['font.size'] = 14 
plt.rcParams['axes.labelsize'] = 12 
plt.rcParams['axes.labelweight'] = 'bold' 
plt.rcParams['axes.titlesize'] = 12 
plt.rcParams['xtick.labelsize'] = 12 
plt.rcParams['ytick.labelsize'] = 12 
plt.rcParams['legend.fontsize'] = 12 
plt.rcParams['figure.titlesize'] = 12 
plt.rcParams['image.cmap'] = 'jet' 
plt.rcParams['image.interpolation'] = 'none' 
plt.rcParams['figure.figsize'] = (16, 8) 
plt.rcParams['lines.linewidth'] = 2 
plt.rcParams['lines.markersize'] = 8
plt.rcParams["axes.grid"] = False

colors = ['xkcd:pale orange', 'xkcd:sea blue', 'xkcd:pale red', 'xkcd:sage green', 'xkcd:terra cotta', 'xkcd:dull purple', 'xkcd:teal', 'xkcd: goldenrod', 'xkcd:cadet blue',
'xkcd:scarlet']
cmap_big = cm.get_cmap('Spectral', 512)
cmap = mcolors.ListedColormap(cmap_big(np.linspace(0.7, 0.95, 256))) 
bbox_props = dict(boxstyle="round,pad=0.3", fc=colors[0], alpha=.5)

In [4]:
import folium
from folium.plugins import MarkerCluster


In [5]:
import geoplot as gplt
import geopandas as gpd
import geoplot.crs as gcrs
import imageio
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import mapclassify as mc
import numpy as np
import mysql.connector

# Importing the Dataset

In [17]:
mydb= mysql.connector.connect(host='localhost', user='root', password='MaestroRoberto'
                             ,database='prova')

In [18]:
mycursor=mydb.cursor()

In [19]:
mycursor.execute('SELECT * FROM tableprova')

In [20]:
myresult=mycursor.fetchall()

In [21]:
mycursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = 'prova' AND TABLE_NAME ='tableprova'")

In [22]:
mytitles=mycursor.fetchall()

In [23]:
mytitles = list(sum(mytitles, ())) 

In [24]:
mytitles

['AverageTemperature',
 'AverageTemperatureUncertainty',
 'City',
 'Country',
 'dt',
 'Latitude',
 'Longitude']

In [25]:
df=pd.DataFrame(myresult)


In [31]:
df.columns=[mytitles[4],mytitles[0],mytitles[1],mytitles[2],mytitles[3],mytitles[-2],mytitles[-1]]

In [32]:
df

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1849-01-01,26.704,1.435,Abidjan,Côte D'Ivoire,5.63N,3.23W
1,1849-02-01,27.434,1.362,Abidjan,Côte D'Ivoire,5.63N,3.23W
2,1849-03-01,28.101,1.612,Abidjan,Côte D'Ivoire,5.63N,3.23W
3,1849-04-01,26.14,1.3869999999999998,Abidjan,Côte D'Ivoire,5.63N,3.23W
4,1849-05-01,25.427,1.2,Abidjan,Côte D'Ivoire,5.63N,3.23W
...,...,...,...,...,...,...,...
239172,2013-05-01,18.979,0.807,Xian,China,34.56N,108.97E
239173,2013-06-01,23.522,0.647,Xian,China,34.56N,108.97E
239174,2013-07-01,25.251,1.042,Xian,China,34.56N,108.97E
239175,2013-08-01,24.528,0.84,Xian,China,34.56N,108.97E


In [33]:
df['dt']=pd.to_datetime(df.dt)

In [34]:
df['year'] = pd.DatetimeIndex(df['dt']).year
df['month'] = pd.DatetimeIndex(df['dt']).month

In [35]:
df=df[df['year']>1950]

In [36]:
df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,year,month
1224,1951-01-01,26.944000000000006,0.441,Abidjan,Côte D'Ivoire,5.63N,3.23W,1951,1
1225,1951-02-01,27.118000000000002,0.569,Abidjan,Côte D'Ivoire,5.63N,3.23W,1951,2
1226,1951-03-01,27.983,0.274,Abidjan,Côte D'Ivoire,5.63N,3.23W,1951,3
1227,1951-04-01,27.914,0.204,Abidjan,Côte D'Ivoire,5.63N,3.23W,1951,4
1228,1951-05-01,26.69,0.168,Abidjan,Côte D'Ivoire,5.63N,3.23W,1951,5


# Display the cities on a WorldMap

In [37]:
world_map= folium.Map()


In [38]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Piero")


In [39]:
CITIES=df.City.drop_duplicates().tolist()

In [40]:
LAT=[]
LONG=[]
for city in CITIES:
    location = geolocator.geocode(city)
    LAT.append(location.latitude)
    LONG.append(location.longitude)

In [41]:
N_CITY=[]
for city in CITIES:
    N_CITY.append(len(df[df['City']==city]))

In [42]:
N_CITY[0:5]

[753, 753, 753, 753, 753]

In [43]:
count=0
for i in N_CITY:
    if i==N_CITY[0]:
        count+=1
if count==len(N_CITY):
    print('All the cities have the same instances, and they are ' + str(N_CITY[0]))

All the cities have the same instances, and they are 753


In [44]:
CITIES=np.array(CITIES)

In [45]:
LAT=np.repeat(np.array(LAT),753)
LONG=np.repeat(np.array(LONG),753)

In [46]:
df['Latitude']=LAT
df['Longitude']=LONG

<ipython-input-46-6c47a21c9aed>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Latitude']=LAT
<ipython-input-46-6c47a21c9aed>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Longitude']=LONG


In [47]:
df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,year,month
1224,1951-01-01,26.944000000000006,0.441,Abidjan,Côte D'Ivoire,5.320357,-4.016107,1951,1
1225,1951-02-01,27.118000000000002,0.569,Abidjan,Côte D'Ivoire,5.320357,-4.016107,1951,2
1226,1951-03-01,27.983,0.274,Abidjan,Côte D'Ivoire,5.320357,-4.016107,1951,3
1227,1951-04-01,27.914,0.204,Abidjan,Côte D'Ivoire,5.320357,-4.016107,1951,4
1228,1951-05-01,26.69,0.168,Abidjan,Côte D'Ivoire,5.320357,-4.016107,1951,5


In [48]:
city_data=df.drop_duplicates(subset='City')

In [49]:
marker_cluster = MarkerCluster().add_to(world_map)

for i in range(len(city_data)):
        lat = city_data.iloc[i]['Latitude']
        long = city_data.iloc[i]['Longitude']
        radius=5
        folium.CircleMarker(location = [lat, long], radius=radius,fill =True).add_to(marker_cluster)

In [50]:
world_map


In [51]:
MIS_LIST=list(df[df['AverageTemperature']==''].City.drop_duplicates())

In [52]:
NOT_MIS_LIST=list(df.City.drop_duplicates())

In [53]:
list1,list2=MIS_LIST,NOT_MIS_LIST
difference = list(set(list1).symmetric_difference(set(list2)))


In [54]:
print ('The cities that has 0 missing values are: \n')
for i in range(len(difference)):
    print(difference[i] + '\n')

The cities that has 0 missing values are: 

Toronto

New York

Mexico

Santo Domingo

Chicago

Los Angeles

Montreal



# Missing values

In [55]:
df['AverageTemperature']= df.AverageTemperature.apply(pd.to_numeric, args=('coerce',))


<ipython-input-55-5af3a8ce4c1a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['AverageTemperature']= df.AverageTemperature.apply(pd.to_numeric, args=('coerce',))


In [56]:
print ('The total number of missing temperature in the dataset is ' + str(sum(df.AverageTemperature.isna())) + ' out of '+str(len(df)))

The total number of missing temperature in the dataset is 93 out of 75300


In [57]:
df['AverageTemperatureUncertainty']= df.AverageTemperatureUncertainty.apply(pd.to_numeric, args=('coerce',))


<ipython-input-57-06f252500d71>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['AverageTemperatureUncertainty']= df.AverageTemperatureUncertainty.apply(pd.to_numeric, args=('coerce',))


In [58]:
print ('The total number of missing error in the dataset is ' + str(sum(df.AverageTemperatureUncertainty.isna())) + ' out of '+str(len(df)))

The total number of missing error in the dataset is 93 out of 75300


In [59]:
print('The cities that present missing values are ' + str(len(MIS_LIST)))

The cities that present missing values are 93


In [60]:
MIS_INDEX=[]
for i in range(len(MIS_LIST)):
    
    mis_data=df[df['City']==MIS_LIST[i]].reset_index()
    index = mis_data['AverageTemperature'].index[mis_data['AverageTemperature'].apply(np.isnan)]
    MIS_INDEX.append(index.tolist()[0])

In [61]:
MIS_INDEX[0:5]

[752, 752, 752, 752, 752]

In [62]:
count=0
for i in MIS_INDEX:
    if i==MIS_INDEX[0]:
        count+=1
if count==len(MIS_INDEX):
    print('All the cities have the same missing value, and it is the number ' + str(MIS_INDEX[0]))

All the cities have the same missing value, and it is the number 752


In [63]:
df=df.reset_index()

In [64]:
df=df.drop(columns=['index'])

In [65]:
df=df.dropna()

In [66]:
df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,year,month
0,1951-01-01,26.944,0.441,Abidjan,Côte D'Ivoire,5.320357,-4.016107,1951,1
1,1951-02-01,27.118,0.569,Abidjan,Côte D'Ivoire,5.320357,-4.016107,1951,2
2,1951-03-01,27.983,0.274,Abidjan,Côte D'Ivoire,5.320357,-4.016107,1951,3
3,1951-04-01,27.914,0.204,Abidjan,Côte D'Ivoire,5.320357,-4.016107,1951,4
4,1951-05-01,26.690,0.168,Abidjan,Côte D'Ivoire,5.320357,-4.016107,1951,5


In [67]:
df.to_csv('ClimateData.csv')